In [1]:
pip install opencv-python numpy matplotlib tensorflow keras


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Insert this at the top of your notebook (Cell 1 or 2)

import tensorflow as tf

# 1. Enable Eager execution (recommended for debugging and simple Keras models)
tf.config.run_functions_eagerly(True)

# 2. Enable tf.data debug mode (Call without arguments)
tf.data.experimental.enable_debug_mode()

print("TensorFlow Eager Execution and Debug Mode Enabled.")

TensorFlow Eager Execution and Debug Mode Enabled.


In [3]:
# Cell 1: Imports, Constants, and Model Architecture

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint 
import tensorflow as tf # Required for explicit eager execution fix

# CRITICAL FIX: Ensure eager execution is enabled globally
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

# --- Define Constants and Paths ---
DATA_DIR = 'fer2013'  
MODEL_FILE = 'emotion_model_fer2013.h5'
CASCADE_FILE = 'haarcascade_frontalface_default.xml'

IMAGE_SIZE = (48, 48)
BATCH_SIZE = 64
NUM_CLASSES = 7
GLOBAL_EPOCHS = 10 

# --- Model Architecture (Function Definition) ---
def build_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(48, 48, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

print("Libraries, constants, and model architecture initialized.")

Libraries, constants, and model architecture initialized.


In [4]:
# Cell 2: Data Loading using flow_from_directory

# 1. Initialize Data Generators (with training augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255) # Only normalize for test/validation

# 2. Load Training Data
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'train'),
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# 3. Load Test/Validation Data
validation_generator = test_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'test'),
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False 
)

# Extract emotion labels (folder names)
emotion_labels = list(train_generator.class_indices.keys())
np.save('emotion_labels.npy', np.array(emotion_labels)) 
print(f"\nEmotion Labels: {emotion_labels}")

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.

Emotion Labels: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [5]:
# Cell 3: Build, Compile, and Conditional Training (Skips Training if Model Exists)

# --- Initialize Model and Check for Saved Weights ---
model = None

if os.path.exists(MODEL_FILE):
    # 🧠 Load: File found, load weights, architecture, and optimizer state.
    print(f"--- 🧠 Found saved model '{MODEL_FILE}'. Loading model. ---")
    model = load_model(MODEL_FILE)
    
    # CRITICAL FIX: Recompile the model to reset the optimizer's state 
    # This prevents the "Unknown variable" ValueError when model.fit is called later.
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
else:
    # 🚀 BUILD: File NOT found, build and compile a new model.
    print("--- 🚀 No saved model found. Building new CNN architecture from scratch. ---")
    model = build_cnn_model() # Uses function defined in Cell 1

    # Compile Model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

model.summary()

# --- Conditional Training ---

# This condition ensures model.fit() ONLY runs if the model file is MISSING.
if not os.path.exists(MODEL_FILE):
    
    # Setup Model Checkpoint
    checkpoint = ModelCheckpoint(
        MODEL_FILE, 
        monitor='val_accuracy', 
        verbose=1, 
        save_best_only=True, 
        mode='max'
    )
    callbacks_list = [checkpoint]
    
    print(f"\nStarting training for {GLOBAL_EPOCHS} epochs...")

    history = model.fit(
        train_generator,
        epochs=GLOBAL_EPOCHS, 
        validation_data=validation_generator,
        callbacks=callbacks_list,
        initial_epoch=0
    )
    
    print(f"\nModel successfully trained and saved as '{MODEL_FILE}'.")
    
else:
    # This executes when the model is found, and training is skipped.
    print("\nTraining skipped: Model file found. Proceeding to Cell 4 (Live Detection).")

--- 🧠 Found saved model 'emotion_model_fer2013.h5'. Loading model. ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     9,437,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,534,855 (36.37 MB)

 Trainable params: 9,534,407 (36.37 MB)

 Non-trainable params: 448 (1.75 KB)


Training skipped: Model file found. Proceeding to Cell 4 (Live Detection).


In [6]:
# Cell 4: Real-Time Live Detection

# --- Load Resources ---
try:
    live_model = load_model(MODEL_FILE)
    face_classifier = cv2.CascadeClassifier(CASCADE_FILE)
    loaded_emotion_labels = np.load('emotion_labels.npy', allow_pickle=True)
except Exception as e:
    print(f"Error loading required files for live detection: {e}")
    live_model = None 

if live_model:
    # --- Initialize Video Stream ---
    cap = cv2.VideoCapture(0)  
    if not cap.isOpened():
        print("Webcam access failed. Check hardware or permissions.")
    else:
        print("Webcam initialized. Look for the window. Press 'q' to quit.")
        
        # --- Main Real-Time Detection Loop ---
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
                roi_gray = gray[y:y+h, x:x+w]
                cropped_img = cv2.resize(roi_gray, IMAGE_SIZE)
                
                # Preprocess for CNN
                model_input = cropped_img.astype('float32') / 255.0
                model_input = np.expand_dims(model_input, axis=0)
                model_input = np.expand_dims(model_input, axis=-1)

                # Predict the emotion
                predictions = live_model.predict(model_input, verbose=0)
                max_index = np.argmax(predictions[0])
                predicted_emotion = loaded_emotion_labels[max_index]
                
                # Display the prediction
                cv2.putText(frame, str(predicted_emotion), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            cv2.imshow('Facial Emotion Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # --- Cleanup ---
        cap.release()
        cv2.destroyAllWindows()
else:
    print("Live detection was skipped due to an issue loading the model or dependencies.")

Webcam initialized. Look for the window. Press 'q' to quit.
